#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [2]:
using Random, LinearAlgebra
p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
 1.25  0.25  0.0  0.0   0.25  0.25  0.0   0.0   0.0  0.0
 0.25  1.25  0.0  0.0   0.25  0.25  0.0   0.0   0.0  0.0
 0.0   0.0   1.0  0.0   0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0  1.25  0.0   0.0   0.25  0.25  0.0  0.25
 0.25  0.25  0.0  0.0   1.25  0.25  0.0   0.0   0.0  0.0
 0.25  0.25  0.0  0.0   0.25  1.25  0.0   0.0   0.0  0.0
 0.0   0.0   0.0  0.25  0.0   0.0   1.25  0.25  0.0  0.25
 0.0   0.0   0.0  0.25  0.0   0.0   0.25  1.25  0.0  0.25
 0.0   0.0   0.0  0.0   0.0   0.0   0.0   0.0   1.0  0.0
 0.0   0.0   0.0  0.25  0.0   0.0   0.25  0.25  0.0  1.25

In [3]:
n = 1000 #Sample size

Random.seed!(1234)

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Sn = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  1.19918       0.272751   -0.0275862    …   0.00764566    0.083198
  0.272751      1.2777     -0.072279        -0.0511888    -0.0736561
 -0.0275862    -0.072279    1.02957          0.000292289   0.0221311
  0.0254728     0.0108026   0.00487371      -0.00744628    0.207586
  0.224212      0.319557   -0.0213081        0.0469386    -0.0155358
  0.250244      0.298127   -0.037503     …  -0.0257465     0.0936313
 -0.000599258  -0.0153294  -0.0328447       -0.0662836     0.264067
 -0.00627049   -0.0882766   0.0259089       -0.0113125     0.163798
  0.00764566   -0.0511888   0.000292289      1.01279      -0.00581329
  0.083198     -0.0736561   0.0221311       -0.00581329    1.32199

In [4]:
# show(stdout, "text/plain", Sn)

In [5]:
# [x1 x2]

In [6]:
# [k, k]

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [7]:
include("algorithm2.jl")

findmultPCs_deflation (generic function with 1 method)

In [8]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Sn, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

In [ ]:
# show(stdout, "text/plain", x_best)

#### Step 3: R/C++ implementation

Applies the R code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [ ]:
using RCall

R"""library(sPCAmPC)"""
# R"""
# library(devtools)
# reload(pkg = "sPCAmPC", quiet = FALSE)"""


In [ ]:
R"""

TestMat <- $Sn 

results <- cpp_findmultPCs_deflation(TestMat, 2, c(4, 4), numIters=20)
"""

In [ ]:
R"""
results 
"""

In [14]:
@rget results

10×2 Matrix{Float64}:
 0.0   0.0
 0.5   0.0
 0.0   0.5
 0.5   0.0
 0.0   0.0
 0.5   0.0
 0.0  -0.5
 0.0  -0.5
 0.5   0.0
 0.0   0.5

In [13]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
  0.0  -0.5
  0.0   0.5
  0.0  -0.5
  0.0   0.0
 -0.5   0.0
  0.0  -0.5
  0.0   0.0
  0.5   0.0
 -0.5   0.0